# NB #1:  Pool Exploration 

#### Pool address: 0x8b6e6E7B5b3801FEd2CaFD4b22b8A16c2F2Db21a  -- 80% WETH - 20% DAI

The Balancer Pool cadCAD model provides a core infrastructure for simulating Balancer Pools in the **Token Engineering design and verification process**.

It allows to simulate all types of Balancer Pool transactions and verify new designs and application cases.  
In this notebook we plug **historical on-chain data** to the cadCAD model, and analyse **Pool Power** and **Pool Characteristics**.

For more information on simulating Balancer AMM Pools check out the documentation available here [Balancer Simulations documentation](xxxadd link).

## Table of content
(XX - to be added)

# A. System Context

### System Specification
- Differential Syntax Diagram (XX update!!)
- Link to Mathematical Specification (XX link Documentation/.py for reference)
- Link to Software Archtitecture (XX link!)

### Naming Convention

All code provided for this package follows a specific naming convention, as documented here (link)


# B. cadCAD Notebook

### B1.1 Dependencies

In [1]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

# C. Run Simulations 

### C1.1 Initialize Pool


(XX how to data parse in readme.file in data folder)  
(XX state variables, make list)  
(XX Parameters, make list)  
(XX weigth, check, why 0.1/0.4, translate to decimal?)  
(XX print the pool address?)

In [6]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from decimal import Decimal
parameters = {
    'spot_price_reference': ['DAI'],
    'decoding_type': ['CONTRACT_CALL']
}
pp.pprint(parameters)

{'decoding_type': ['CONTRACT_CALL'], 'spot_price_reference': ['DAI']}


#### Import Pool Transactions and Initialize Pool state

Use (XX .py name) to write a .json file and use historial transaction data in this model. For more information please visit (XX link).

In [7]:
from model.genesis_states import generate_initial_state

initial_values = generate_initial_state(initial_values_json='data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-initial_pool_states-prices.json', spot_price_base_currency=parameters['spot_price_reference'][0])

#### State Variables and Initial Values

XX - Comments on pool parameters like swap fee, how to change it (atm pulled from .json/Google Cloud)

In [ ]:
print('## State Variables')
print('# Pool')
pool = initial_values['pool']
pp.pprint(initial_values)


#### External Token Prices

In order to analyse USD-based pool metrics like TVL, you can plug in external price feeds. For more information please visit (XX link)

In [ ]:
print('# External token prices, initial state')
token_prices = initial_values['token_prices']
pp.pprint(token_prices)

print('# Action Type')
action_type = initial_values['action_type']
pp.pprint(action_type)


### C1.2 State Update Functions & Policies

cadCAD state update functions replicate the following Balancer Pool Transactions:  
(XX check list, all transaction types, move to documentation?)

a) Add Liquidity
= join policy 
- `p_join_pool`
- `p_join_swap_extern_amount_in`

b) Withdraw Liquidity
= exit policy 
- `p_exit_swap_extern_amount_out`

b) Swap
= swap policy 
- `p_swap_exact_amount_in`

See 'system_policies.py'.  
For a detailed description of the transactions, please visit (Gitbook link)

### C1.3 Partial State Update Blocks


1. Parse action and update pool
2. Update external prices
3. Calculate metrics

The BPool smart contract logic is split in 2, the state update blocks 1 (apply BMath to update pool state) and 3 (use BMath to `get_spot_price` of the tokens after the trades, which is a system metric)



Defined in [partial_state_update_block.py](./model/partial_state_update_block.py)

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks('data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-actions-prices.json')
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### C1.4 Configuration

As stated in C1.2 a pool's state is updated by
- **actions** (such as swaps) and
- **price signals** (USD values of tokens)

These updates are captured in unique **timesteps**.

In most cases you might want to run the simulation across all timesteps included in your .json file - however you can specify any simulation range below.

(XX - instructions to change parameters, what parameters we don't touch atm/documentation?)

In [ ]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps - 1267203 is last action timestep (timestamp - initial timestamp)
        'M': parameters,  # simulation parameters
    }
)

### C1.5 Execution

In [ ]:

from model.sim_runner import *

df = run(initial_values, partial_state_update_blocks, sim_config)

### C1.6 Simulation Output Preparation

Post-processing (utils.py) adds metrics to the data frame, such as 
- `token_k_values`  
based on
- `token_k_balances`
- `token_k_price`  
and calculates accumulated values, such as
- `token_total_value` 


In [ ]:
from model.parts.utils import post_processing

p_df = post_processing(df, include_spot_prices=False)

p_df.head(1000)

# D. Simulation Outcome & Pool Exploration

Below we show a range of plots exploring pool states in the simulation:

**a) Pool Power:**  
- D1.1 TVL (Total Value Locked, over time) compared to  
- D1.1 Pool Size Growth (Number of tokens in the pool, over time)  
- D1.2 Token Balances (individual balances, over time)

**b) Pool Characteristics:**  
- D1.3 Source of Pool Power Growth (Pool Shares (BPT) vs. Fees collected, over time)
- D1.4 Token Ratio (over time)
- D1.5 Action Types (per timestep)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# read csv, for test purpose only
# p_df = pd.read_csv('BAMM-out.csv', sep=";")
# p_df

### D1.1 TVL & Pool Size Growth

In [ ]:
#TVL vs. Pool Size Growth
tvl_p_df = p_df[['timestep', 'token_dai_price', 'token_weth_price']] #change columns to tvl +  total balance
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(go.Scatter(x=tvl_p_df['timestep'],y=tvl_p_df['token_weth_price'], line=dict(color='#F79F34'), name="TVL total_token_value"), secondary_y=False,)
fig.add_trace(go.Scatter(x=tvl_p_df['timestep'],y=tvl_p_df['token_dai_price'], line=dict(color='#015B99'), name="total_token_balance"), secondary_y=True,)
#Layout
fig.update_layout(title_text="<b>TVS vs. Pool Size Growth</b>")
fig.update_xaxes(title_text="timestep")
fig.update_yaxes(title_text="<b>TVL total_token_value</b> in USD", secondary_y=False)
fig.update_yaxes(title_text="<b>total_token_balance</b> in #", secondary_y=True)
fig.show()  

**Observation:**
- TVL grows, while
- Pool Size declines in observation period  
**This pool's TVL increases solely due to the growth of USD value of the underlying tokens!**

### D1.2 Token Balances

In [ ]:
k = 2 #define number of tokens in your pool
fig = make_subplots(rows=k, cols=1)

fig.add_trace(go.Scatter(x=p_df['timestep'], y=p_df['token_weth_balance'], name='token_weth_balance'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df['timestep'], y=p_df['token_dai_balance'], name='token_dai_balance'), row=2, col=1)
fig.update_layout(height=400, width=1000, title_text="<b>Token Balances in #</b>")
fig.show()

### D1.3 Sources of Growth 

In [ ]:
#Pool Shares
ps_p_df = p_df[['timestep','pool_shares']]
fig = px.line(ps_p_df, x=ps_p_df['timestep'],y=ps_p_df['pool_shares'])
fig.update_layout(height=300, width=1000, title_text="<b>Pool Shares (BPT)</b>")
fig.update_xaxes(title_text="timestep")
fig.update_yaxes(title_text="<b>pool shares</b> in #", range=[99.995,100.15])
fig.show()

In [ ]:
# fees to be added!

### D1.4 Token Ratio (over time)

In [ ]:
# to be added!

### D1.5 Action Types

In [ ]:
#update with final dataframe and "total_token_balance y-axis"
fig = px.scatter(p_df, x='timestep', y='token_weth_balance', color='action_type')
fig.update_layout(height=400, width=1000, title_text="<b>Action Type / Timestep</b>")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

# E. System Validation and Limitations

(XX check all and add!)

- move to Gitbook!
- document the steps taken to validate if the model reflects real Balancer AMM properly (Did we build the right model?)
- document the steps taken to verify if the model creates reliable results (Did we build the model right?)

### Notes (Draft!)
**a) BMath Calculations:**  
Our goal is to implement the BMath calculations in this Python model in a way that it replicates *exactly* the calculation results in an EVM.

We've verified the model with a series of tests:
- create tests using balancer's smart contract repos
- generate a pool contract in a local EVM, do a swap or whatever operation, 
- put those input outputs as a test in python, port the code, test to see if the results match

**b) external USD price feed** 
- in this simulation we're using historical USD prices from xxx (source)  
- to map blocks and transactions we've ... (how we parsed USD price feed)

**c) Our simulation does not include:**  
- gas prices or add_fees when adding liquidity  
- 

**Results: (summarize)**

(Notes for ourselves:
- assertAlmostEqual takes 7 decimal places for comparision, sometimes we had to set 5 decimal places for the test to pass
- we could publish the EVM tests  as companion in the docs later but right now is very rough code
- to run the test go to your virtual env, instlal requirements with pip, then run pytest
- they should pass
- when everything is tested and works as the contracts, we could move on to cadCAD stuff

According to Balancer.finance documentation "The formulas are sufficient to describe the functional specification, but they are not straightforward to implement for the EVM, in part due to a lack of mature fixed-point math libraries." (https://docs.balancer.finance/core-concepts/protocol/index))

# F. Comments

closing comments if appropriate, and links to other notebooks/other use cases